# Data modeling at scale: introduction to dimension modeling
_Diego Ardila - Staff Data Scientist at Shopify_

* [Data modeling at scale?](#data_modeling_at_scale)
  * [What is data warehouse?](#why_a_data_warehouse)
  * [Why a data warehouse?](#what_is_data_warehouse)
* [What is ETL?](#what_is_etl)
* [Methodology (dimensional modeling)](#dimensional_modeling)
  * [Example 1: Who are the best movie directors?](#dimensional_modeling_example_1)
    * [Why dimensional modeling](#why_dimensional_modeling_example_1)
    * [Dimensions](#dimensional_modeling_dimensions)
    * [Facts](#dimensional_modeling_facts)
  * [Example 2: Whhen did a show started, and ended? What was the total runtime?](#dimensional_modeling_example_2)

## Data modeling at scale? <a class="anchor" id="data_modeling_at_scale"></a>

*Infrastructure (ETL/ELT) + Methodology (dimensional modeling)  = Data warehouse*

## What is data warehouse? <a class="anchor" id="what_is_data_warehouse"></a>

1. Database/Datalake 
2. Data collected from one or multiple sources
3. Designed to support reporting and data analytics

## Why a data warehouse? <a class="anchor" id="why_a_data_warehouse"></a>
1. Keep historic data
2. Central view
3. Data quality
4. Common model
5. Query performance without impacting production systems
6. Augment source system  

## What is ETL? <a class="anchor" id="what_is_etl"></a>

**Extraction + Transformation + Loading**

- Extract: move data from the source to the data lake. Technlogies such as: iceberg, kafka  
- Transformation: augment/transform the sources into a consistent format. Technologies such as Spark, DBT, Flink, Apache Beam.   
- Load: load data into frontroom tables for consumption. Technologies such as BigQuery, Trino 

All this typically wrapped around some scheduling solution, e.g. Airflow, Luigi, Prefect. 

Alternatively, we can talk about ELT


## Methodology (dimensional modeling) <a class="anchor" id="dimensional_modeling_example_2"></a>
- Data warehouse designed technique, developed by [Kimball and Ross](https://www.kimballgroup.com/data-warehouse-business-intelligence-resources/books/data-warehouse-dw-toolkit/)
- Based on the "star" model. Two main elements: conformed dimensions and facts:

![alt text](https://cdn.holistics.io/guidebook/star-schema.png "Title")

- Four large steps: 
  - Select the business process
  - Identify grain
  - Identify the dimensions
  - Identify facts
  - Implement model. We can use a SQL transformation engine. Check out [DBT](https://www.getdbt.com/).
- Let's use an example to introduce the main concepts


### Our database: IMDB database.




[IMDB datasets](https://www.imdb.com/interfaces/)

![alt text](imdb_simplified.png )



### Example 1: Who are the best movie directors? <a class="anchor" id="dimensional_modeling_example_1"></a>

#### Why dimensional modeling? <a class="anchor" id="why_dimensional_modeling_example_1"></a>

--Let's do first the SQL query

``` -- SQL
with directors as (
  select  tconst, d as director
  from crew c, unnest(directors) d
),
ratings_directors as (
select d.director, avg(averageRating) as averageRating
from 
    titles t 
    join directors d on t.tconst = d.tconst
    join ratings r on t.tconst = r.tconst
group by 1
)
select d.director, averageRating
from ratings_directors
order by averageRating
limit 10 desc
```

This works, so what are the problems? 

- We carry the naming and standards of the source database
- Okay to perform on ad-hoc basis, but what if you have to regularly use this information?
- Agregations in production are costly. You don't want to do this

Ideally we have something simpler, 

```
select director_id, avg(average_rating) as average_rating
from directors_facts join directors_dimension using(director_id)
group by 1
order by 2
limit 10 desc
```

#### Dimensions <a class="anchor" id="dimensional_modeling_dimensions"></a>

- Business entity. In our example, `directors_dimension`
- There are several types of dimensions:
  - Type 1 dimension
  - Type 2 dimension
- Simple case: type 1 dimension:

```
create table directors_dimension as
select distinct d as director_id
from crew c, unnest(directors) d


create table movie_dimension as
select t.tconst as movie_id, primaryTitle as primary_title, startYear as release_year, runtime_minutes as runtime_minutes
from titles t

```

#### Fact tables <a class="anchor" id="dimensional_modeling_facts"></a>

- Table containing measurements
- Grain defined by related dimensions
- Facts are usually additive, but not always
- Resolves many to many relationships
- There are several types of fact tables
  - *Transaction Fact Table* represents an event that occurs at the primary point.
  - *Snapshot Fact Table* describes the state of things at a particular time.
  - *Accumulated Fact Table* is used to show the activity of a process that has a beginning and an end. 
- Simple case: Transaction Fact Table of movies directed by a director

```
-- What is the grain?
create table directors_facts as
with directors as (
  select  tconst, d as director_id
  from crew c, unnest(directors) d
),
select d.director_id, t.tconst as movie_id, avg(averageRating) as average_movie_rating
from 
    titles t 
    join ratings r on t.tconst = r.tconst
    join directors d on t.tconst = d.tconst
```



Can we do? 

```
select director_id, avg(average_rating) as average_rating
from directors_facts join directors_dimension using(director_id)
group by 1
order by 2 desc
limit 10 desc
```

Yes!

##### What if we have a new use case: first movie published by a director publis?

```
select director_id, min_by(release_year, primary_title) as first_movie
from directors_facts 
    join directors_dimension using(director_id)
    join movies_dimension using(movie_id)
group by 1
order by 1
limit 10 desc
```

### Example 2: When did a show started and ended? What was the total runtime? <a class="anchor" id="dimensional_modeling_example_2"></a>

1. What grain?: tv show
2. What kind of query are we looking for? 

```
select show_id, first_episosde_started_on, last_episode_ended_on, number_of_episodes, number_of_seasons, total_run_time
from show_accumulating_facts
```

3. Let's do first the SQL query

``` -- SQL
with shows as (
  select  tconst as show_id, primaryTitle as primary_title, startYear as release_year
  from titles t
  where titleType = 'tvseries'
),
shows_facts as (
select s.show_id
    , min(e.startYear) as first_episosde_started_on
    , max(e.startYear) as last_episode_ended_on
    , count(distinct seasonNumber) as number_of_seasons
    , count(distinct episodeNumber) as number_of_episodes
    , sum(total_run_time) as total_run_time
from 
    shows s 
    left join titles e on s.show_id = e.parentTconst
 group by 1
)
select show_id, first_episosde_started_on, last_episode_ended_on, number_of_episodes, number_of_seasons, total_run_time
from shows_facts
```

4. Enters dimensional modeling....


##### Which dimensions?

```
create table shows_dimension as
select  tconst as show_id, primaryTitle as primary_title, startYear as release_year
 from titles t
where titleType = 'tvseries'

```

```
create table shows_accumulating_facts as
with shows_facts as (
select s.show_id
    , min(e.startYear) as first_episosde_started_on
    , max(e.startYear) as last_episode_ended_on
    , count(distinct seasonNumber) as number_of_seasons
    , count(distinct episodeNumber) as number_of_episodes
    , sum(total_run_time) as total_run_time
from 
    shows_dimension s 
    left join titles e on s.show_id = e.parentTconst
 group by 1
)
select show_id, first_episodes_started_on, last_episodes_ended_on, number_of_episodes, number_of_seasons, total_run_time
from shows_facts

```


### Example 3: What was the first and last shows of a director? What was the best and worst shows? How many shows has a director had?  <a class="anchor" id="dimensional_modeling_example_3"></a>

1. What grain?: tv show
2. What kind of query are we looking for?

```
select director_id
    , first_show_started_on
    , last_show_started_on
    , fsd.show_id as best_show_id
    , lsd.show_id as last_show_id
    , number_of_shows
from directors_accumulating_facts d
    join shows_dimension fsd on  d.best_show_id = fsd.show_id
    join shows_dimension lsd on  d.last_show_started_on = lsd.show_id
```
3. Let's do first the SQL query 
4. Which dimensions? Which facts?



# Other topics that we did not covered


- Slow changing dimensions
- Junk dimensions
- Outrigger dimension
- Role playing dimensions
- Bus Architecture
- ... :)  